In [1]:
import pandas as pd
import mysql.connector


In [1]:
# Data Ingestion Notebook
import pandas as pd
import mysql.connector

print("Notebook initialized correctly")


Notebook initialized correctly


In [ ]:
df = pd.read_csv("../data/jobs.csv")
len(df)


3755

In [3]:
def infer_primary_skill(job_title):
    title = job_title.lower()
    if "machine learning" in title or "ml engineer" in title:
        return "Machine Learning"
    elif "data scientist" in title:
        return "Python"
    elif "data analyst" in title:
        return "SQL"
    elif "ai" in title or "deep learning" in title:
        return "Deep Learning"
    else:
        return "Other"

df["primary_skill"] = df["job_title"].apply(infer_primary_skill)


In [4]:
conn = mysql.connector.connect(
    host="localhost",
    user="job_user",
    password="job_pass",
    database="job_market_db"
)
cursor = conn.cursor()


In [5]:
companies_df = df[['company_location', 'company_size']].drop_duplicates()

cursor.executemany(
    "INSERT IGNORE INTO companies (company_location, company_size) VALUES (%s, %s)",
    list(companies_df.itertuples(index=False, name=None))
)

conn.commit()


In [6]:
skills_df = df[['primary_skill']].drop_duplicates()

cursor.executemany(
    "INSERT IGNORE INTO skills (skill_name) VALUES (%s)",
    [(s,) for s in skills_df["primary_skill"]]
)

conn.commit()


In [7]:
companies_db = pd.read_sql("SELECT * FROM companies", conn)
skills_db = pd.read_sql("SELECT * FROM skills", conn)

company_map = {
    (row.company_location, row.company_size): row.company_id
    for _, row in companies_db.iterrows()
}

skill_map = {
    row.skill_name: row.skill_id
    for _, row in skills_db.iterrows()
}


C:\Users\lssar\AppData\Local\Temp\ipykernel_4080\3031094363.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  companies_db = pd.read_sql("SELECT * FROM companies", conn)
C:\Users\lssar\AppData\Local\Temp\ipykernel_4080\3031094363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  skills_db = pd.read_sql("SELECT * FROM skills", conn)


In [8]:
insert_job = """
INSERT INTO jobs (
    job_title, experience_level, employment_type,
    work_year, salary_in_usd, remote_ratio,
    company_id, skill_id
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

for _, row in df.iterrows():
    cursor.execute(
        insert_job,
        (
            row["job_title"],
            row["experience_level"],
            row["employment_type"],
            int(row["work_year"]),
            int(row["salary_in_usd"]),
            int(row["remote_ratio"]),
            company_map[(row["company_location"], row["company_size"])],
            skill_map[row["primary_skill"]],
        )
    )

conn.commit()


In [9]:
pd.read_sql("SELECT COUNT(*) AS total_jobs FROM jobs", conn)


C:\Users\lssar\AppData\Local\Temp\ipykernel_4080\3823357366.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT COUNT(*) AS total_jobs FROM jobs", conn)


,total_jobs
0,11265


In [10]:
cursor.close()
conn.close()


In [11]:
print("Data ingestion completed successfully")


Data ingestion completed successfully
